<a href="https://colab.research.google.com/github/OneFineStarstuff/Pinn/blob/main/Complete_MCTS_Example_(Tic_Tac_Toe).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random

class Node:
    def __init__(self, state, player, parent=None):
        self.state = state          # Board state as a tuple of 9 cells
        self.player = player        # Player to move at this node ('X' or 'O')
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0

    def ucb(self, c=1.41):
        if self.visits == 0:
            return float('inf')
        return (self.value / self.visits) + c * math.sqrt(math.log(self.parent.visits) / self.visits)

def check_winner(state):
    wins = [(0,1,2),(3,4,5),(6,7,8),
            (0,3,6),(1,4,7),(2,5,8),
            (0,4,8),(2,4,6)]
    for a,b,c in wins:
        if state[a] != ' ' and state[a] == state[b] == state[c]:
            return state[a]
    if ' ' not in state:
        return 'Draw'
    return None

def transition_fn(state, player):
    """Generate all possible next states for the current player."""
    next_states = []
    for i in range(9):
        if state[i] == ' ':
            new_state = list(state)
            new_state[i] = player
            next_states.append(tuple(new_state))
    return next_states

def rollout_fn(state, player):
    """Play random moves until terminal state."""
    current_player = player
    current_state = list(state)
    while True:
        winner = check_winner(current_state)
        if winner:
            if winner == 'Draw':
                return 0
            return 1 if winner == player else -1
        moves = [i for i in range(9) if current_state[i] == ' ']
        move = random.choice(moves)
        current_state[move] = current_player
        current_player = 'O' if current_player == 'X' else 'X'

def eval_fn(state, player):
    """Heuristic evaluation: +1 win, -1 loss, 0 draw/ongoing."""
    winner = check_winner(state)
    if winner == player:
        return 1
    elif winner == 'Draw':
        return 0
    elif winner is not None:
        return -1
    return 0  # Non-terminal: neutral

def backpropagate(node, result):
    """Alternate perspective at each step."""
    while node:
        node.visits += 1
        node.value += result
        result = -result  # Switch perspective
        node = node.parent

def mcts(root_state, player, iterations=1000, c=1.41):
    root = Node(root_state, player)

    for _ in range(iterations):
        node = root

        # 1. Selection
        while node.children and all(child.visits > 0 for child in node.children):
            node = max(node.children, key=lambda n: n.ucb(c))

        # 2. Expansion
        next_states = transition_fn(node.state, node.player)
        if next_states:
            unvisited_states = [s for s in next_states if not any(c.state == s for c in node.children)]
            if unvisited_states:
                state = random.choice(unvisited_states)
                next_player = 'O' if node.player == 'X' else 'X'
                child = Node(state, next_player, parent=node)
                node.children.append(child)
                node = child

        # 3. Simulation / Evaluation
        result = rollout_fn(node.state, node.player)

        # 4. Backpropagation
        backpropagate(node, result)

    # Return the move with the most visits
    best_child = max(root.children, key=lambda n: n.visits)
    return best_child.state

# -------------------------
# Demo: Play as 'X' vs MCTS
# -------------------------
if __name__ == "__main__":
    board = tuple([' '] * 9)
    current_player = 'X'

    while True:
        winner = check_winner(board)
        if winner:
            print("Winner:", winner)
            break

        if current_player == 'X':
            # Human move
            print("\nBoard:")
            for i in range(0, 9, 3):
                print(board[i:i+3])
            move = int(input("Enter your move (0-8): "))
            if board[move] != ' ':
                print("Invalid move, try again.")
                continue
            board = list(board)
            board[move] = 'X'
            board = tuple(board)
        else:
            # AI move
            board = mcts(board, current_player, iterations=500)
        current_player = 'O' if current_player == 'X' else 'X'